In [14]:
import pandas as pd
from pandas.plotting import radviz
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
#import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm, preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn import linear_model
from sklearn.ensemble import BaggingClassifier, GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [15]:
def read(file):
    return pd.read_csv(file)

#features.set_index(['Index'], inplace = True)
#features.drop(columns = ['Index'], inplace = True)

features = read('Pmax_backward_Final.csv')
features.head(5)

,α [°],H [mm],b [mm],Ls [mm],d [mm],γ [N/mm3],Pmax [N],Vroot [mm3],DR [ ]
0,15,90.0,19.4114,17.555563,5.0,0.000016,9.080644,4534.350557,0.8
1,15,90.0,12.9410,41.703709,5.0,0.000016,8.758713,3535.876732,0.8
2,15,90.0,19.4114,17.555563,5.0,0.000016,11.753734,8267.564868,0.8
3,15,90.0,12.9410,41.703709,5.0,0.000016,10.797683,5796.469487,0.8
4,15,90.0,19.4114,17.555563,5.0,0.000016,14.095079,23281.011643,0.8


In [16]:
#summary statistics
features.describe()
features.columns.values

array(['α [°]', 'H [mm]', 'b [mm]', 'Ls [mm]', 'd [mm]', 'γ [N/mm3]',
       'Pmax [N]', 'Vroot [mm3]', 'DR [ ]'], dtype=object)

In [17]:
#pd.options.display.max_rows = 181
pd.options.display.max_columns = 30
#features.sort_values(by = "L [mm]", ascending = True, inplace = True)
#display(features)

In [18]:
#one hot encoding
pd.options.display.max_columns = 181
pd.options.display.max_rows = 181

if 'n [count]' in features.columns and 'Material' in features.columns:
    features = pd.get_dummies(features, columns = ['n [count]', 'Material'])
elif 'n [count]' in features.columns:
    features = pd.get_dummies(features, columns = ['n [count]'])
elif 'Material' in features.columns:
    features = pd.get_dummies(features, columns = ['Material'])
#features.head(5)
#display(features)
#print_shape(features)

In [19]:
#target pmax/f
pmax = np.array(features['Pmax [N]']).reshape(1, -1)

#Remove labels from the features
features = features.drop('Pmax [N]', axis = 1)
feature_list = list(features.columns)
features = StandardScaler().fit_transform(features)
pmax = Normalizer().fit_transform(pmax).reshape(-1, 1)
#pca = PCA(n_components = 11)
#principalComponents = pca.fit_transform(features)
#features = pd.DataFrame(data = principalComponents, columns = ['pc1', 'pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','pc11'])
#display(features)
#print(pca.explained_variance_ratio_.cumsum())

#Covert to numpy arrays
features = np.array(features)


In [20]:
#split the data into training set and testing set 
train_features, test_features, train_labels, test_labels = train_test_split(features, pmax, test_size = 0.25, random_state = 42)

In [21]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (132, 8)
Training Labels Shape: (132, 1)
Testing Features Shape: (45, 8)
Testing Labels Shape: (45, 1)


In [22]:
gb = GradientBoostingRegressor(random_state=42, 
                               n_estimators=1000, 
                               learning_rate=0.1, 
                               max_features=2, 
                               min_samples_leaf=3)
gb.fit(train_features, train_labels.ravel())

GradientBoostingRegressor(max_features=2, min_samples_leaf=3, n_estimators=1000,
                          random_state=42)

In [23]:
gb_predictions = gb.predict(test_features)
print(gb_predictions)

[0.03903951 0.03117068 0.02339287 0.08904034 0.02195669 0.06557666
 0.02061221 0.09678759 0.02085682 0.02629087 0.04552322 0.04074962
 0.04373921 0.01370424 0.03581345 0.03029896 0.02950791 0.05337357
 0.0269638  0.06167888 0.10367039 0.04726877 0.0831254  0.10126403
 0.103416   0.02758364 0.10774698 0.04590988 0.16012977 0.05043656
 0.05796501 0.02494322 0.04497294 0.01671531 0.09195365 0.04311282
 0.08542735 0.01616656 0.02102596 0.06968236 0.10376186 0.02929496
 0.08271131 0.02680014 0.05725017]


In [24]:
gb_mean_absolute_error = mean_absolute_error(test_labels, gb_predictions)
print('Mean Absolute Error:', round(gb_mean_absolute_error, 5))

Mean Absolute Error: 0.00484


In [25]:
mean_square_error = mean_squared_error(test_labels, gb_predictions, squared=False)

print("Mean Square Error: ", round(mean_square_error, 5))

Mean Square Error:  0.00706


In [26]:
gb_explained_variance_score = explained_variance_score(test_labels, gb_predictions)

print("Explained variance score: ", round(gb_explained_variance_score, 7))

Explained variance score:  0.9638057
